In [6]:
import chess, chess.svg
from eval import evaluate
import math
from collections import namedtuple
import os
import pandas as pd
from search import search, searchAB
import torch
import numpy as np
import pickle
from timeit import timeit
board = chess.Board()


In [17]:
PIECE_TABLE = {
    'P' : (0,1),
    'p' : (1,-1),
    'N' : (2,1),
    'n' : (3,-1),
    'B' : (4,1),
    'b' : (5,-1),
    'R' : (6,1),
    'r' : (7,-1),
    'Q' : (8,1),
    'q' : (9,-1),
    'K' : (10,1),
    'k' : (11,-1)
     
}
COLOR_TABLE = {
    'w' : 1,
    'b' : -1
}

In [44]:
def b2b():
    out = np.zeros(768, dtype=np.float32)
    for sq, p in board.piece_map().items():
        sm = p.symbol()
        idx = PIECE_TABLE[sm][0] * 64 + sq
        out[idx] = PIECE_TABLE[sm][1] * (board.turn + (1 - board.turn) * -1)
    return 

In [45]:
timeit(b2b, number=1000000)

23.42040800000541

In [209]:
ran_ge = np.arange(64, dtype=np.uint64)[::-1] # range exists as a built-in. sorry
b_boards = np.array([n for n in range(1,12,2)])
w_boards = np.array([n for n in range(0,11,2)])
def b2b2(board: chess.Board):
    p = np.array([board.pawns, board.knights, board.bishops, board.rooks, board.queens, board.kings], dtype=np.uint64)
    pcolor = p[:, None] & np.array([board.occupied_co[1], board.occupied_co[0]], dtype=np.uint64)
    pcolor = pcolor.flatten()
    bitboard = (pcolor[:, None] & (1 << ran_ge)) > 0
    bitboard = bitboard.astype(np.float32)
    bitboard[w_boards] = bitboard[w_boards] * (board.turn + (1 - board.turn) * -1) # Negate white bits if black to move
    bitboard[b_boards] = bitboard[b_boards] * (board.turn * -1 + (1 - board.turn) ) # Negate black bits if white to move
    return torch.from_numpy(bitboard.flatten())

In [132]:
timeit(b2b2, number=1000000)


11.986394416999246

In [20]:
from nn.model import nnEval_model_set1_v1, nnEval_model_set1_v2, BigNet, SmallNet
from evalnn import nnEval_fn_set1_v1, nnEval_fn_set1_v2, board2bb_set1_v1, board2bb_set1_v2

In [4]:
checkpoint = "./nn/lightning_logs/version_14/checkpoints/epoch=4-step=126545.ckpt"
evalobj = nnEval_model_set1_v1.load_from_checkpoint(checkpoint, net=BigNet(768))
eval_net = evalobj.net
eval_func = Evalnn_fn(eval_net, board2bb)

In [ ]:
board.push(searchAB(eval_func, board, 4)[1])
board

In [ ]:
board.push(chess.Move.from_uci('f6e4'))
board

In [29]:
smallnet_chkpt = './nn/lightning_logs/version_17/checkpoints/epoch=19-step=506180.ckpt'
smallnet_test = nnEval_model_set1_v1.load_from_checkpoint(smallnet_chkpt, net=SmallNet(768))
smallnet = smallnet_test.net
small_eval = nnEval_fn_set1_v1(smallnet, board2bb_set1_v1)

In [4]:
board = chess.Board()   

In [87]:
board.pop()

Move.from_uci('f4d6')

In [ ]:
evaluation, move = searchAB(small_eval, board, 5)
board.push(move)
print(evaluation)
board


In [15]:
test = chess.Board()

In [5]:
v2_chkpt = './nn/lightning_logs/version_18/checkpoints/epoch=19-step=506180.ckpt'
v2_test = nnEval_model_set1_v2.load_from_checkpoint(v2_chkpt, net=SmallNet(768))
v2net = v2_test.net
v2_eval = nnEval_fn_set1_v2(v2net, board2bb_set1_v2)

In [7]:
v2_eval(test)

tensor([68.3250], grad_fn=<MulBackward0>)

In [ ]:
evaluation, move = searchAB(v2_eval, board, 5)
board.push(move)
print(evaluation)
board


In [49]:
np.arange(8)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [50]:
def getindices():
    RANGE = np.arange(64, dtype=np.uint64)
    p_vals = np.arange(10)
    piece_bitboards = np.array([board.pawns, board.knights, board.bishops, board.rooks, board.queens], dtype=np.uint64)
    piece_bitboards = piece_bitboards[:, None] & np.array([board.occupied_co[1], board.occupied_co[0]], dtype=np.uint64)
    piece_bitboards = piece_bitboards.flatten()
    piece_bitboards = (piece_bitboards[:, None] & (1 << RANGE)) > 0

    piece_indices = (piece_bitboards * p_vals[:, None] * 64) + RANGE
    w_indices = piece_indices + (board.king(1)) * 64 * 10
    w_indices = w_indices * piece_bitboards
    b_indices = piece_indices + (board.king(0)) * 64 * 10
    b_indices = b_indices * piece_bitboards
    return w_indices[w_indices != 0], b_indices[b_indices != 0]

In [40]:
timeit(getindices, number=1000000)

18.062290375000885

In [19]:
from nn.parse import fen2halfKP_sparse, fen2bitboard


In [47]:
fen2halfKP_sparse(board.fen(), 30)

(array([2944, 2689, 2818, 3075, 2821, 2694, 2951, 2568, 2569, 2570, 2571,
        2572, 2573, 2574, 2575, 2672, 2673, 2674, 2675, 2676, 2677, 2678,
        2679, 3064, 2809, 2938, 3195, 2941, 2814, 3071]),
 array([38784, 38529, 38658, 38915, 38661, 38534, 38791, 38408, 38409,
        38410, 38411, 38412, 38413, 38414, 38415, 38512, 38513, 38514,
        38515, 38516, 38517, 38518, 38519, 38904, 38649, 38778, 39035,
        38781, 38654, 38911]),
 30,
 1)

In [48]:
board.pawns & board.occupied_co[0]

71776119061217280

In [23]:
pth = './nn/Datasets/halfKP_evals_sparse.npz'
f = open(pth, 'rb')
nu = np.load(f, allow_pickle=True)

In [65]:
a = nu['white_feature_indices']
c = nu['black_feature_indices']
b = nu['num_active_features'].astype(int)
d = nu['stms'].astype(int)